# Text Preprocessing and Sentiment analysis of r/Happy

Import and read the csv file 

In [1]:
!pip3 install emoji
!pip3 install nltk
!pip3 install spacy
import pandas as pd
import emoji as emoji
import nltk
from nltk.tokenize import RegexpTokenizer
!python -m spacy download en_core_web_sm
import spacy
import en_core_web_sm
from nltk.stem import WordNetLemmatizer

     |████████████████████████████████| 13.9 MB 2.9 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [16]:
happy_final = pd.read_csv("happy_final.csv")

In [17]:
happy_final

,author,title,selftext,full_link,created_utc,score,num_comments,num_crossposts
0,GoodNaturedGamer,After months of work my work with dnd.Disablit...,NaN,https://www.reddit.com/r/happy/comments/mnq89p...,2021-04-09 20:42:50,1,1,0
1,ASAP-Broccoli,Happy 1st birthday to my best friend,NaN,https://www.reddit.com/r/happy/comments/mnot6q...,2021-04-09 19:32:31,1,2,0
2,cctrjkrfan,Neighbors moved out of the apartment complex a...,NaN,https://www.reddit.com/r/happy/comments/mnoo7x...,2021-04-09 19:25:31,1,3,0
3,_keiki_kitten,Lemme see ‘em,NaN,https://www.reddit.com/r/happy/comments/mnoa9w...,2021-04-09 19:06:21,1,2,0
4,Utterlybored,Two Weeks Since Shot #2 Means I Have Reached P...,"...or as I call it, 2V + 14\n\nOff to hug the ...",https://www.reddit.com/r/happy/comments/mnmzct...,2021-04-09 18:02:50,1,3,0
...,...,...,...,...,...,...,...,...
535,Link_Worried,She saved a whole colony of bees. What a legend.,NaN,https://www.reddit.com/r/happy/comments/m42hlv...,2021-03-13 08:45:42,1,3,0
536,Emilena_,Officially Diagnosed with ADHD and Thrilled!!!,I feel like this has been a moment I have been...,https://www.reddit.com/r/happy/comments/m41bbi...,2021-03-13 07:09:58,1,6,0
537,Emilena_,Officially diagnosed with ADHD today!,[removed],https://www.reddit.com/r/happy/comments/m419v2...,2021-03-13 07:06:55,1,2,0
538,carpet_fire,Jacket says sup,NaN,https://www.reddit.com/r/happy/comments/m40kfp...,2021-03-13 06:13:04,1,2,0


We start the sentiment analysis with text preprocessing. 

Store all titles in a list

In [18]:
Title_all = []
for i in happy_final.loc[:,"title"]:
    Title_all.append(i)
#print(Title_all)
print('Total titles scraped:', len(Title_all))

Total titles scraped: 540


Then, we preprocess the titles to get rid of the emojis, stopwords, and links 

In [19]:
#Make all titles into a string object  
List = Title_all
List = [str(i) for i in List] #this makes every title into a string itself but separated by commas 
string_uncleaned = ",". join(List) #join all the strings separated by commas 
#string_uncleaned

In [20]:
#removing emojis

def give_emoji_free_text(text): 
    return emoji.get_emoji_regexp().sub(r'', text)

emoji_free = give_emoji_free_text(string_uncleaned)

#emoji_free


We tokenize the words. Tokenizing breaks apart every word in the string into an individual word which would then carry it’s own “positive” or “negative” sentiment based on our sentiment analyzer later.

In [21]:
tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+?!!!')
tokenized_string = tokenizer.tokenize(emoji_free)
#print(tokenized_string)

Some words are uppercase while others are lowercase. We must standardize to all lower case then put it in a list 

In [22]:
lower_case = [word.lower() for word in tokenized_string]

#print(lower_case)

In [23]:
#removing stop words
import en_core_web_sm
nlp = en_core_web_sm.load()
#spacy.load("en_core_web_sm")
all_stopwords = nlp.Defaults.stop_words

text = lower_case 
no_stopwords = [i for i in text if not i in all_stopwords]

#print(no_stopwords)

Our last step in preprocessing the text is to lemmatize the words. Lemmatization allows us to query the text and returns the root/base word. 

Ex: running, ran, runs are all forms of the base word run

In [24]:
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

lemmatized = [lemmatizer.lemmatize(word) for word in no_stopwords]

#print(lemmatized)


[nltk_data] Downloading package wordnet to /Users/gabychu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [25]:
## printing len to see how much we have cut down 
print("number of words originally:", len(string_uncleaned))
print("number of words after taking out emojis:", len(emoji_free))
print("number of words after tokenization:", len(tokenized_string))
print("number of words after taking out stopwords:", len(no_stopwords))
print("number of words after lemmatization", len(lemmatized))
#lets store the final output into a new variable 
cleaned = lemmatized

number of words originally: 49129
number of words after taking out emojis: 49002
number of words after tokenization: 9356
number of words after taking out stopwords: 4579
number of words after lemmatization 4579


In [26]:
df = pd.DataFrame(cleaned, columns=["word"])
df.to_csv('cleaned_happy_data.csv', index=False)

Now that we have a cleaned output, we are going to use the VADER model (Valence Aware Dictionary for Sentiment Reasoning) to calculate the polarity score (valence) of each word